In [ ]:
import librosa
import librosa.display
import IPython.display as ipd
import stumpy 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import math
import os
import sys
import logging
logging.basicConfig(level=logging.INFO)

In [ ]:
current = os.path.dirname(os.path.realpath(sys.argv[0]))
parent = os.path.dirname(current)
sys.path.append(parent)

from MSig import Motif, NullModel

In [ ]:
# Load the audio file
audio_path = '../data/audio/imblue.mp3'
ipd.Audio(audio_path)

In [ ]:
#Extract MFCCs
y, sr = librosa.load(audio_path) #22050 sr
n_mfcc = 12
n_fft = int(sr * 0.046)  # 46 milliseconds STFT window
hop_length = int(sr * 0.023)  # 23 milliseconds STFT hop
X = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft, n_mels=32)
X = X.astype(np.float64)
X.shape

In [ ]:
data = pd.DataFrame(X.T)
data['datetime'] = data.index * hop_length / sr
data['datetime'] = pd.to_datetime(data['datetime'], unit='s')
data = data.set_index('datetime')
data


In [ ]:
#Visualize MFCCs
plt.figure(figsize=(12,4))
librosa.display.specshow(X, sr=sr, x_axis='time')
plt.colorbar(format="%+2f")
plt.show()

In [ ]:
# discover motifs
include = None
normalize = True
subsequence_lengths = [int(secs * sr / hop_length) for secs in [1,3,5,10]]

In [ ]:
# for m in subsequence_lengths:
#     mp, mp_indices = stumpy.mstump(X, m, include=include, normalize=normalize)
#     np.save('../results/audio/mp/mp_normalized={}_include={}_m={}.npy'.format(normalize,include,m), mp)
#     np.save('../results/audio/mp_indices/mp_indices_normalized={}_include={}_m={}.npy'.format(normalize,include,m), mp_indices)

In [ ]:
def table_summary_motifs(motif_indices, motif_distances, motif_subspaces, data, m, normalize, max_allowed_dist, excl_zone):
    mp_stats_table = pd.DataFrame(columns=["ID", "k", "Features", "m", "#Matches", "Indices", "max(dists)", "min(dists)", "med(dists)"])

    motif_index = 0

    n_vars, n_time = data.shape

    if normalize:
        data = (data - np.mean(data, axis=1)[:, np.newaxis]) / np.std(data, axis=1)[:, np.newaxis]
    
    model_empirical = NullModel(data, model="empirical")
    #model_gaussian_empirical = NullModel(data, model="kde")
    #model_gaussian_theoretical = NullModel(data, model="gaussian_theoretical")

    for motif_indice, match_indices in enumerate(motif_indices):

        dimensions = motif_subspaces[motif_indice]
            
        #remove filling values of -1 and Nans from motif_indices and match_distances
        match_indices = match_indices[match_indices != -1]
        match_distances = motif_distances[motif_indice]
        match_distances = match_distances[~np.isnan(match_distances)]

        #if is empty, skip
        if len(match_indices) == 0:
            continue
        
        #remove trivial matches  
        non_trivial_matches = []
        for indice in match_indices:
           trivial = False
           for indice_new in non_trivial_matches:
               if abs(indice - indice_new) < m/4:
                   trivial = True
                   break
           if not trivial:
               non_trivial_matches.append(indice)
        match_indices = non_trivial_matches

        #max_possible_matches = np.floor((n_time-m)/np.ceil(m/2))+1


        #get the multidim time serie motif in the dimensions
        multivar_subsequence = data[dimensions][:,match_indices[0]:match_indices[0]+m]
    
        max_dist = np.max(match_distances)
        min_dist = np.min(match_distances[1:])
        avg_dist = np.mean(match_distances[1:])
        std_dist = np.std(match_distances[1:])
        med_dist = np.median(match_distances[1:])
        
        #D is distance profile between the motif pattern and Time serie
        if max_allowed_dist is None:
            D = np.empty((n_time-m+1, len(dimensions)))
            for i, dimension in enumerate(dimensions):
                D[:,i] = stumpy.mass(multivar_subsequence[i], data[dimension], normalize=normalize)
            D = np.mean(D, axis=1)
            max_allowed_dist = np.nanmax([np.nanmean(D) - 2.0 * np.nanstd(D), np.nanmin(D)])
            

        #data features are now the ones in the dimensions
        used_features = [f"{dimension}" for dimension in dimensions]

        # dist_media_elemento = SQRT((dist^2)/n_length)
        #max_delta = max_allowed_dist        # dist_max do motif = dist_max de um elemento (worst case) max_dist = sqrt(max_delta^2) <=> max_delta = max_dist
        max_delta = math.sqrt(max_allowed_dist**2/m) 
        delta_thresholds = [max_delta]*len(data)

        
        
        #########SIG#########
        motif = Motif(multivar_subsequence, dimensions, delta_thresholds, len(match_indices))
        p = motif.set_pattern_probability(model_empirical, vars_indep=True)
        pvalue = motif.set_significance(n_time-m-1, n_vars, idd_correction=False) 
        ######
        #p = motif.set_pattern_probability(model_gaussian_empirical, vars_indep=True)
        #pvalue = motif.set_significance(n_time-excl_zone, n_vars, idd_correction=False)
        ######
        #p = motif.set_pattern_probability(model_gaussian_theoretical, vars_indep=True)
        #pvalue = motif.set_significance(n_time-excl_zone, n_vars, idd_correction=False)

        stats_df = {"ID": str(motif_index), "k":len(dimensions),
                    "Features":",".join(used_features),
                        "m":m,
                    "#Matches": len(match_indices)-1,
                        "Indices":match_indices,
                        "max(dists)": np.around(max_dist,3), "min(dists)": np.around(min_dist,3),
                        "med(dists)": np.around(med_dist,3),  "P": p, "p-value": pvalue}
    
        mp_stats_table = pd.concat(
            [mp_stats_table, pd.DataFrame.from_records([stats_df])], ignore_index=True)
        
        motif_index += 1
    return mp_stats_table

In [ ]:
k = None
min_neighbors = 1
cutoffs = np.inf
max_matches = 99999
average_delta = 0.5
max_dists = []
max_motifs = 99999
mp_stats_table = pd.DataFrame()
for m in subsequence_lengths:
    max_distance = math.sqrt(m)*average_delta
    max_dists.append(max_distance)

    excl_zone = int(np.ceil(m/4))
    mp= np.load('../results/audio/mp/mp_normalized={}_include={}_m={}.npy'.format(normalize,include,m))
    mp_indices = np.load('../results/audio/mp_indices/mp_indices_normalized={}_include={}_m={}.npy'.format(normalize,include,m))
    motif_distances, motif_indices, motif_subspaces, motif_mdls = stumpy.mmotifs(X, mp, mp_indices, max_distance=max_distance,max_matches=max_matches,
                                                                                 cutoffs=cutoffs, min_neighbors=min_neighbors, max_motifs=max_motifs, k=k, include=include, normalize=normalize)
    
    if len(motif_indices[0]) == 0:
        continue
       
    print("m:{}, #Motifs:{}".format(m, len(motif_indices)))  
    table = table_summary_motifs(motif_indices, motif_distances, motif_subspaces, X, m, normalize, max_distance, excl_zone)
    print("Sig ", np.sum(table["p-value"] < 0.01))

    #hochberg procedure
    p_values = table["p-value"].to_numpy()
    critical_value =  NullModel.hochberg_critical_value(p_values, 0.05)
    sig = table["p-value"] < critical_value if critical_value != 0 else table["p-value"] <= critical_value
    table["Sig_Hochber"] = sig

    print("Sig after Hochberg: {}, critical value: {}".format(np.sum(sig), critical_value))


    mp_stats_table = pd.concat([mp_stats_table, table], ignore_index=True)

mp_stats_table.to_csv('../results/audio/table_motifs_min_neighbors={}_max_distance={}_cutoffs={}_max_matches={}_max_motifs={}.csv'.format(min_neighbors, max_dists, cutoffs, max_matches, max_motifs), index=False)

In [ ]:
#create a new table for each motif length with statistics of the motifs (number of motifs found,
# number of significant motifs, average number of matches +- std, average of features +- std, 
#average probability +- std, average pvalue +- std)
mp_stats_table = pd.read_csv('../results/audio/table_motifs_min_neighbors={}_max_distance={}_cutoffs={}_max_matches={}_max_motifs={}.csv'.format(min_neighbors, max_dists, cutoffs, max_matches, max_motifs))

motif_lengths = mp_stats_table["m"].unique()
motif_stats_table = pd.DataFrame(columns=["m", "#motifs" , "avg_n_matches", "avg_n_features",  "avg_probability",  "avg_pvalue", "#sig_motifs(<0.01)", "#sig_hochberg"])
for m in motif_lengths:
    table = mp_stats_table[mp_stats_table["m"] == m]
    n_motifs = table.shape[0]
    n_sig_motifs_001 = table[table["p-value"] < 0.01].shape[0]
    n_sig_motifs_hochberg = table[table["Sig_Hochber"] == True].shape[0]
    avg_n_matches = round(table["#Matches"].mean(),2), round(table["#Matches"].std(),3)
    avg_n_features = round(table["k"].mean(),2), round(table["k"].std(),3)
    avg_probability = table["P"].mean(), table["P"].std()    
    avg_pvalue = table["p-value"].mean(), table["p-value"].std()

    stats_df = {"m": m, "#motifs": n_motifs, "#sig_motifs(<0.01)": n_sig_motifs_001, "#sig_hochberg": n_sig_motifs_hochberg,
                "avg_n_matches": avg_n_matches, "avg_n_features": avg_n_features, "avg_probability": avg_probability, "avg_pvalue": avg_pvalue}
    motif_stats_table = pd.concat([motif_stats_table, pd.DataFrame.from_records([stats_df])], ignore_index=True)

motif_stats_table

In [ ]:
motif_stats_table_print = motif_stats_table.copy()
motif_stats_table_print["avg_n_matches"] = motif_stats_table["avg_n_matches"].apply(lambda x: "{:.2f} +- {:.2f}".format(x[0], x[1]))
motif_stats_table_print["avg_n_features"] = motif_stats_table["avg_n_features"].apply(lambda x: "{:.2f} +- {:.2f}".format(x[0], x[1]))
motif_stats_table_print["avg_probability"] = motif_stats_table["avg_probability"].apply(lambda x: "{:.2e} +- {:.2e}".format(x[0], x[1]))
motif_stats_table_print["avg_pvalue"] = motif_stats_table["avg_pvalue"].apply(lambda x: "{:.2e} +- {:.2e}".format(x[0], x[1]))
print(motif_stats_table_print.to_latex(index=False))

In [ ]:
#sort by score unified
mp_stats_table = mp_stats_table.sort_values(by="p-value", ascending=True)
mp_stats_table

In [ ]:
#get top 5 most significant for each motif length
for m in motif_lengths:
    top_5_motifs = mp_stats_table[mp_stats_table["m"] == m].head(5)
    print(top_5_motifs.to_latex(index=False, columns=["ID", "k", "Features", "#Matches", "max(dists)", "min(dists)", "med(dists)", "P", "p-value"]))
    print("\n\n")

In [ ]:
def plot_motif(ts_list, features,  m, motif_indexes, motif_name):

    fig, axes = plt.subplots(ncols=2, nrows=len(ts_list), figsize=(10, 3*len(ts_list)), squeeze=False)
    for i in range(0,len(ts_list)):
        ts = ts_list[i]
        #plot light grey
        axes[i,1].plot(ts, color='black', linewidth=0.5, alpha=0.5)

        colors = plt.cm.tab20(np.linspace(0, 1, len(motif_indexes)))
        axes[i,0].set_prop_cycle('color', colors)
        axes[i,1].set_prop_cycle('color', colors)

        for index in motif_indexes:
            subsequence_match = ts.iloc[index:index+m]
            #original motif in the next plot with the same color
            axes[i,0].plot(subsequence_match.values) 
            # highlight the motif in the original time serie
            axes[i,1].plot(subsequence_match, linewidth=2)
        
        plt.setp(axes[i,0].xaxis.get_majorticklabels(), rotation=90)
        #remove x labels and ticks except from last plot
        if i != len(ts_list)-1:
            axes[i,0].axes.get_xaxis().set_visible(False)
            axes[i,1].axes.get_xaxis().set_visible(False)

        plt.setp(axes[i,0].xaxis.get_majorticklabels(), rotation=90)


        #format the x axis to show the time and rotate for better reading
        axes[i,1].xaxis.set_major_formatter(mdates.DateFormatter('%M:%S'))
        plt.setp(axes[0,1].xaxis.get_majorticklabels(), rotation=45)
        axes[i,0].set_ylabel(features[i], rotation=90, size='large')


    #title of the fig
    axes[0,0].set_title("Raw Subsequences")
    axes[0,1].set_title("Motif in TS")
    plt.tight_layout()
    plt.savefig('../results/audio/m='+str(m)+'_motif_'+str(motif_name)+".pdf",bbox_inches='tight')
  
    return None

In [ ]:
#plot motifs
for m in motif_lengths:
    print("Motif length: ", m)
    top_motifs = mp_stats_table[mp_stats_table["m"] == m].sort_values(by="p-value")
    for top_motif in top_motifs.to_dict(orient="records"): 
        features = top_motif["Features"].split(",")
        ts_list = [data[int(feature)] for feature in features]
        indices = top_motif['Indices'].replace("[","").replace("]","").split(",")
        indices = [int(i) for i in indices]
        motif_name = top_motif["ID"]
        start = indices[0]
        end = start + m
        segment = y[start*hop_length:end*hop_length]
        #print in seconds and datetime format
        print("start: {}, time: {}".format(start, datetime.timedelta(seconds=librosa.core.frames_to_time(start, sr=sr, hop_length=hop_length))))
        
        display(ipd.Audio(segment, rate=sr))
        #display nearest match
        nearest_match = indices[1]
        end_match = nearest_match + m
        segment_match = y[nearest_match*hop_length:end_match*hop_length]
        print("nearest match: {}, time: {}".format(nearest_match, datetime.timedelta(seconds=librosa.core.frames_to_time(nearest_match, sr=sr, hop_length=hop_length))))
        display(ipd.Audio(segment_match, rate=sr))

        #plot znormalized motif and match 
        motif = X[:,start:end]
        match = X[:,nearest_match:end_match]
        plt.figure(figsize=(12,4))
        plt.subplot(1,2,1)
        librosa.display.specshow(motif, sr=sr, x_axis='time')
        plt.colorbar(format="%+2f")
        plt.title("Motif")
        plt.subplot(1,2,2)
        librosa.display.specshow(match, sr=sr, x_axis='time')
        plt.colorbar(format="%+2f")
        plt.title("Match")
        plt.show()


        plot_motif(ts_list, features, m, indices, motif_name)

    
    